In [1]:
# Dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs
from selenium.webdriver.support.ui import WebDriverWait
import re
import pandas as pd
import os
import time
from splinter import Browser
from urllib.request import urlopen
import requests

In [2]:
# Website URL
url = "https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume"

In [3]:
# Website URL Page takes 6 Seconds to load
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
html = "https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume"
browser.visit(html)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [4]:
# Website hmtl page title (Wait for 6 Secs for the browser to open)
title = soup.title.text
print(title)

OTC Markets | Market Activity - Current Market


In [5]:
# Print formatted version of the soup
print(soup.text)

OTC Markets | Market Activity - Current Marketwindow.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){return r(t[n][1][e]||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(t){}}var o,i=t("ee"),a=t(20),c={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(c.console=!0,-1!==o.indexOf("dev")&&(c.dev=!0),-1!==o.indexOf("nr_dev")&&(c.nrDev=!0))}catch(t){}c.nrDev&&i.on("internal-error",function(t){r(t.stack)}),c.dev&&i.on("fn-err",function(t,e,n){r(n.stack)}),c.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(c,function(t,e){return t}).join(", ")))},{}],2:[function(t,e,n){function r(t,e){var n=e?null:c.now();o("err",[t,n])}var o=t("handle"),i=t(21),a=t("ee"),c=t("loader"),s=t("gos"),f=window.onerror,u=!1,d="

In [6]:
# Extract Most Active Stocks html class
most_active_stocks = soup.find("div", {"class": "_12doApv4EC"}).text
most_active_stocks.split(" ")

['Displaying', '25', 'of', '18509', 'Most', 'Active']

In [7]:
# Each page contains 25 Records, therefore index [1] = # of Displayed Records, Index [3] = Total Number of Displayed Records
# displayed_records represent the Number of Displayed Records (index [1])
displayed_records = int(most_active_stocks.split(" ")[1])
displayed_records

25

In [8]:
# total_displayed_records represent the Total Number of Displayed Records (Index [3])
total_displayed_records = int(most_active_stocks.split(" ")[3])
total_displayed_records

18509

In [9]:
# After testing the website several times, the best number you can extract is 8,000 Records 
# where each page contains 25 records, therefore 4000/25 = 160 Pages 
# Once the driver opens the page 25 Records
records = 4000 
total_pages = int(records/25) 
total_pages

160

In [57]:
# Request to open the browser and wait for 6 secs

driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get(url)
time.sleep(6)

In [58]:
soup_loop = BeautifulSoup(html,'html.parser')

In [59]:
# Press the "More" button if found, class_name _2sFaw3zGf1


for x in range(1, total_pages):
        
        wait = WebDriverWait(driver, 30)

        python_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "_2sFaw3zGf1")))
        ActionChains(driver).move_to_element(python_button).click().perform()


In [65]:
soup_loop = BeautifulSoup(driver.page_source,'html.parser')

In [66]:
# Find all div class "_2_IqcDGekT" to look for the TIER (Logo Column)
## Loop through the Table and append to the OTC
### Click on the More button to open all the stock pages (More Button 'Class="_12doApv4EC"')  _2sFaw3zGf1
#### Add "most_active_stocks_no" instead of "5" (Each page has 25 Records, Range 5 stops at 150 records) 

# TIER List contains the continas the following columns qx, qb, ps
TIER_list = []

# OTC List  contains the following columns SYMBOL, PRICE, % CHANGE, $ VOL, SHARE_VOL & TRADES
OTC_list = []

tier_column = soup_loop.findAll("div", {"class": "_2_IqcDGekT"})

table_rows = soup_loop.find_all('tr')

# Loop through the page and append records in the OTC_list & TIER_Lits

for t in tier_column:
    TIER_list.append(t.img["src"][12:14])
    
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    OTC_list.append(row)

In [75]:
# Convert TIER List to Dataframe
TIER = pd.DataFrame(TIER_list, columns=["TIER"])
TIER.head()

,TIER
0,ps
1,ps
2,ps
3,qx
4,qx


In [76]:
# Re-number the index
TIER.index += 1
TIER.head()

,TIER
1,ps
2,ps
3,ps
4,qx
5,qx


In [77]:
# Convert the OTC List into Dataframe
OTC = pd.DataFrame(OTC_list, columns=["SYMBOL", "PRICE", "Per_CHANGE", "VOL", "SHARE_VOL", "TRADES"])
OTC.head()

,SYMBOL,PRICE,Per_CHANGE,VOL,SHARE_VOL,TRADES
0,None,None,None,None,None,None
1,TCEHY,49.76,+1.10,"100,474,741","2,015,981","5,064"
2,IDEXY,14.70,+0.62,"56,714,077","3,872,068",302
3,SFTBY,55.88,+4.84,"48,413,077","875,543","2,476"
4,RHHBY,33.23,+0.97,"46,887,527","1,415,561","1,369"


In [78]:
# Dropping the first OTC df row
OTC = OTC.iloc[1:]
OTC.head()

,SYMBOL,PRICE,Per_CHANGE,VOL,SHARE_VOL,TRADES
1,TCEHY,49.76,+1.10,"100,474,741","2,015,981","5,064"
2,IDEXY,14.70,+0.62,"56,714,077","3,872,068",302
3,SFTBY,55.88,+4.84,"48,413,077","875,543","2,476"
4,RHHBY,33.23,+0.97,"46,887,527","1,415,561","1,369"
5,GBTC,7.325,+6.93,"45,268,895","6,058,170","7,443"


In [80]:
# Count
TIER.count()

TIER    4000
dtype: int64

In [81]:
# Count 
OTC_Markets.count()

TIER          4000
SYMBOL        4000
PRICE         4000
Per_CHANGE    4000
VOL           4000
SHARE_VOL     4000
TRADES        4000
dtype: int64

In [82]:
# Merge the two Dataframe together
OTC_Markets = pd.merge(TIER, OTC, left_index=True, right_index=True)
OTC_Markets.head()

,TIER,SYMBOL,PRICE,Per_CHANGE,VOL,SHARE_VOL,TRADES
1,ps,TCEHY,49.76,+1.10,"100,474,741","2,015,981","5,064"
2,ps,IDEXY,14.70,+0.62,"56,714,077","3,872,068",302
3,ps,SFTBY,55.88,+4.84,"48,413,077","875,543","2,476"
4,qx,RHHBY,33.23,+0.97,"46,887,527","1,415,561","1,369"
5,qx,GBTC,7.325,+6.93,"45,268,895","6,058,170","7,443"


In [86]:
# Remove OTC Pink (ps) & OTC (stop), which is the lowest and most speculative tier 
OTC_Markets_final= OTC_Markets[(OTC_Markets.TIER == "qb") | (OTC_Markets.TIER == "qx")].reset_index(drop=True)
OTC_Markets_final.head()

,TIER,SYMBOL,PRICE,Per_CHANGE,VOL,SHARE_VOL,TRADES
0,qx,RHHBY,33.23,+0.97,"46,887,527","1,415,561","1,369"
1,qx,GBTC,7.325,+6.93,"45,268,895","6,058,170","7,443"
2,qb,FMCKJ,11.25,+2.27,"12,268,924","843,803",293
3,qb,FNMAS,11.315,+1.57,"8,654,586","769,755",695
4,qx,CURLF,10.7564,-3.22,"8,470,204","772,704","2,774"


In [87]:
OTC_Markets_final.index += 1
OTC_Markets_final.head()

,TIER,SYMBOL,PRICE,Per_CHANGE,VOL,SHARE_VOL,TRADES
1,qx,RHHBY,33.23,+0.97,"46,887,527","1,415,561","1,369"
2,qx,GBTC,7.325,+6.93,"45,268,895","6,058,170","7,443"
3,qb,FMCKJ,11.25,+2.27,"12,268,924","843,803",293
4,qb,FNMAS,11.315,+1.57,"8,654,586","769,755",695
5,qx,CURLF,10.7564,-3.22,"8,470,204","772,704","2,774"


In [96]:
# Count
OTC_Markets_final.describe()

,TIER,SYMBOL,PRICE,Per_CHANGE,VOL,SHARE_VOL,TRADES
count,1114,1114,1114,1114,1114,1114,1114
unique,2,538,478,421,536,525,191
top,qb,CWBHF,0.20,0.00,"3,381,427",500,2
freq,634,4,12,77,4,8,46


In [98]:
#export csv to be backup
OTC_qxqb_Markets.to_csv("OTC_Markets_final.csv")

In [102]:
# Import Dataframe into Mongodb
import pandas as pd
from pymongo import MongoClient
import pymongo
import json
import os
import datetime

In [103]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.otc_markets
collection = db.most_active

In [104]:
# Inserr Records in Mongodb
records = json.loads(OTC_Markets_final.T.to_json()).values()
db.most_active.insert_many(OTC_Markets_final.to_dict('records'))